In [ ]:
#install dependencies 
#pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
#pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
#pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
#pip install torch-geometric
#pip install open3d
#pip install pandas
#pip install nibabel
#pip install pyvista
#pip install scikit-image

In [8]:
from pathlib import Path
import glob
import os
import numpy as np
import pandas as pd
from tqdm import tqdm 
import torch
import open3d as o3d

print(torch.__version__)

1.13.1


In [9]:
ids_path = "/vol/space/projects/ukbb/projects/silhouette/eids_filtered.npy"
ids = np.load(ids_path)

In [10]:
ids

array([1000071, 1000180, 1000456, ..., 6023419, 6023786, 6023955])

In [11]:
mesh_path = "/vol/space/projects/ukbb/projects/silhouette/meshes/mesh_v1/3646767/body_surface_mesh.pt"

In [12]:
data_sample = torch.load(mesh_path)

In [13]:
data_sample.edge_index

tensor([[     2,      0,      0,  ..., 255687, 255621, 255688],
        [     1,      3,      1,  ..., 255610, 255614, 255617]])

In [14]:
# with open3d: point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.asarray(data_sample.x))

o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [15]:
# with open3d: triangle mesh 
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

mesh.compute_vertex_normals()
mesh.compute_convex_hull()
o3d.visualization.draw_geometries([mesh],mesh_show_back_face=True,mesh_show_wireframe=True)


[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [16]:
# with open3d: graph
lines = o3d.utility.Vector2iVector(np.asarray(data_sample.edge_index.T))
line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(data_sample.x)),
        lines=lines)

line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines))])

o3d.visualization.draw_geometries([line_set])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
# reading ID mapping
id_map_path = "C:/Users/aorhu/Masaüstü/ADLM/Bridge_eids_60520_87802.csv"

In [ ]:
id_map = pd.read_csv(id_map_path)

In [ ]:
id_map

In [ ]:
# reading basic_features

In [ ]:
basic_features_path = "C:/Users/aorhu/Masaüstü/ADLM/basic_features.csv"

In [ ]:
basic_features = pd.read_csv(basic_features_path)

In [ ]:
basic_features

In [ ]:
#reading file directories for patients into dictionary

In [ ]:
DATA_ROOT = Path("D:/ADLM Data/body_meshes/")

map_dict = {}

In [ ]:
for file in os.listdir(str(DATA_ROOT)):
    _path = os.path.join(str(DATA_ROOT), file).replace('\\', '/')

    sex = "female" if int(basic_features["31-0.0"][basic_features.index[basic_features['eid'] == int(file)]]) == 0 else "male"
    
    map_dict[file] = dict({'path': str(_path) +'/body_surface_mesh.pt',
                           'old_id': int(id_map["eid_60520"][id_map.index[id_map['eid_87802'] == int(file)]]),
                           'sex': sex})

In [ ]:
for file in tqdm(os.listdir(str(DATA_ROOT))):
    _path = os.path.join(str(DATA_ROOT), file).replace('\\', '/')


In [ ]:
map_dict

In [ ]:
len(map_dict)

In [ ]:
def edge_visualize(_index, _dict):
    
    sample = torch.load(_dict[_index]["path"])
    
    lines = o3d.utility.Vector2iVector(np.asarray(sample.edge_index.T))
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(sample.x)),
        lines=lines)

    line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines))])

    o3d.visualization.draw_geometries([line_set])
    
def point_cloud_visualize(_index, _dict):
    
    sample = torch.load(_dict[_index]["path"])
    
    pcd = o3d.geometry.PointCloud(points = o3d.utility.Vector3dVector(np.asarray(sample.x)))

    o3d.visualization.draw_geometries([pcd])

In [ ]:
edge_visualize("1009499", map_dict)

In [ ]:
path = "C:/Users/aorhu/Masaüstü/1000071/spleen_mesh.pt"
sample = torch.load(path)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.asarray(sample.x))

o3d.visualization.draw_geometries([pcd])

In [ ]:
lines = o3d.utility.Vector2iVector(np.asarray(sample.edge_index.T))
line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(sample.x)),
        lines=lines)

line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for i in range(len(lines))])

o3d.visualization.draw_geometries([line_set])

# Mesh Visualization from Segmentation Data


In [17]:
import matplotlib.pyplot as plt
import nibabel as nib
import os
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pyvista
from skimage.measure import marching_cubes
import numpy as np

In [18]:
def pad_edge_list(edges):
    padding = np.ones(edges.shape[0], int)*3
    edges_w_padding = np.vstack((padding, edges.T)).T
    return edges_w_padding

In [19]:
path = "/vol/space/projects/ukbb/projects/silhouette/labels/v1/3646767/body_mask.nii.gz"

In [20]:
body_segment = nib.load(path)
body_segment_data = body_segment.get_fdata()

In [21]:
verts, faces, norms, vals = marching_cubes(body_segment_data, level=0, step_size=1)

In [22]:
verts = verts/np.array(body_segment_data.shape) 

In [23]:
verts

array([[0.00892857, 0.32142857, 0.88429752],
       [0.01339286, 0.32142857, 0.8815427 ],
       [0.01339286, 0.31547619, 0.88429752],
       ...,
       [0.99553571, 0.39285714, 0.88429752],
       [0.99553571, 0.39285714, 0.88705234],
       [0.99553571, 0.39880952, 0.88429752]])

In [24]:
edges = np.concatenate((faces[:,:2], faces[:,1:]), axis=0)

In [25]:
len(faces)

511206

In [28]:
pyvista.set_plot_theme("default")
plotter = pyvista.Plotter(shape=(1, 1), window_size=[1000, 500], border=False)
# plotter = pyvista.Plotter(notebook=False)

plotter.subplot(0, 0)
lines = np.concatenate((np.int32(2*np.ones((edges.shape[0],1))), edges), 1)
mesh = pyvista.PolyData(verts, pad_edge_list(faces))
mesh.lines = lines.flatten()
plotter.add_mesh(mesh, render_points_as_spheres=False, color="lightcoral", show_edges=True, line_width=1, edge_color='mediumorchid', point_size=2, )
plotter.show()

/u/home/zhousiy/.conda/envs/gr_siyu/lib/python3.8/site-packages/pyvista/plotting/plotting.py:148: UserWarning: 
This system does not appear to be running an xserver.
PyVista will likely segfault when rendering.

Try starting a virtual frame buffer with xvfb, or using
  ``pyvista.start_xvfb()``

  warnings.warn(


: 

: 

In [ ]:
mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(np.asarray(verts)),
                                 triangles=o3d.utility.Vector3iVector(np.asarray(faces)))

mesh.compute_vertex_normals()
mesh.compute_convex_hull()
o3d.visualization.draw_geometries([mesh],mesh_show_back_face=True,mesh_show_wireframe=False)
